In [ ]:
import cv2
import imutils
from skimage.filters import threshold_local
from scipy.spatial import distance as dist
import numpy as np
import random as rng
from matplotlib import pyplot as plt
from math import sqrt

In [ ]:
def get_line_coefficients(p1: tuple, p2: tuple):
  x1, y1 = p1
  x2, y2 = p2

  a = y1 - y2
  b = x2 - x1
  c = x1*y2 - x2*y1

  return (a,b,c)


In [ ]:
def get_intersection_points(coeff_1: tuple, coeff_2: tuple):
  a1, b1, c1 = coeff_1
  a2, b2, c2 = coeff_2

  x = 0 
  y = 0

  det = a1 * b2 - a2 * b1

  x_num = b1 * c2 - b2 * c1

  y_num = c1 * a2 - c2 * a1

  # lines are approximately parallel 
  if det > -0.5 and det < 0.5:
    return None
  

  if det != 0:
    x = x_num / det
    y = y_num / det
    return (x, y)
  
  return None


In [ ]:
def distance_between_points(p1: tuple, p2: tuple):
  x1, y1 = p1
  x2, y2, = p2
  distance = sqrt((x2 - x1)**2 + (y2 - y1)**2)
  return distance

In [ ]:
def are_similar_corners(c1: tuple, c2: tuple): 
  return True if distance_between_points(c1, c2) < 100 else False


In [ ]:
def remove_similar_corners(all_corners):
  corners = []
  # get one corner
  for i in range(len(all_corners)): 
    # check with all other corners
    similar_corner = False
    for j in range(len(all_corners)): 
      # if we have checked all corners upto this one then break
      if i == j:
        break
      # if corner is similar to any of the previous corners then we do not add this corner
      if are_similar_corners(all_corners[i], all_corners[j]): 
        similar_corner = True
        break
    if not similar_corner:
      corners.append(all_corners[i])
      
  return corners

In [ ]:
def get_all_possible_corners(coefficients, row_size, col_size): 
  all_corners = []

  # for every combination of coeffs get the intersection points
  for i in range(len(coefficients)):
    for j in range(i, len(coefficients)):
      if(i != j): 
      
        int_point = get_intersection_points(coefficients[i], coefficients[j])
        if int_point != None: 
          x, y = int_point
          # coords should be within the image boundaries
          if x > 0 and y > 0 and x < col_size and y < row_size:
            all_corners.append(int_point)
            
  return all_corners

In [ ]:
def sort_contours(elem):
    return cv2.arcLength(elem, closed=True)
    
def get_corners(grayscale: cv2.Mat, output: cv2.Mat):
  convex_hull_mask = np.zeros((grayscale.shape[0], grayscale.shape[1], 3), dtype=np.uint8)


  convex_hull_mask_grayscale = cv2.cvtColor(convex_hull_mask, cv2.COLOR_BGR2GRAY)

  # Find contours
  contours, _ = cv2.findContours(grayscale, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  # get the largest contour by sorting the contours wrt arcLength
  contours = sorted(contours, key=sort_contours, reverse=True)[:1]

  # convex hull object
  hull_list = []
  hull = cv2.convexHull(contours[-1], True)
  hull_list.append(hull)

  cv2.drawContours(convex_hull_mask_grayscale, hull_list, -1, (255,0,0), 2, 8)

  # cv2.imshow('Convex Hull Mask', convex_hull_mask_grayscale)
  # cv2.waitKey(10)

  # # rho: The resolution parameter rho in pixels.
  # # theta: The resolution of the parameter \theta in radians.
  # # threshold: The minimum number of intersecting points to detecta line.

  lines = cv2.HoughLinesP(image = convex_hull_mask_grayscale, rho = 1, theta = np.pi / 480, minLineLength=200, maxLineGap=640, threshold=640)

  # draw all houghlines
  if lines is not None:
    for line in lines:
      l = line[0]
      cv2.line(output, (l[0], l[1]), (l[2], l[3]), (0,255,0), 5, cv2.LINE_AA )
  
  print("number of lines: ", len(lines))

  if len(lines) >= 4:
    coefficients = []
    for line in lines:
      l = line[0]
      coefficients.append(get_line_coefficients((l[0], l[1]), (l[2], l[3])))
  

    rows, cols = grayscale.shape

    # get all the corners from lines
    all_corners = get_all_possible_corners(coefficients, rows, cols)

    print("all corners found: ", len(all_corners), all_corners)

    print("------------------------------------------------------")

    # remove corners that are similar to one corner
    corners = remove_similar_corners(all_corners)

    print("final corners found: ", len(corners), corners)

    # cv2.imshow('Corners', output)
    # cv2.waitKey(10)
    
    return corners if len(corners) == 4 else None


In [ ]:
def order_points(points: list):
	# initialize a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	pts = np.array(points)
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

In [ ]:
def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	WIDTH = 480
	HEIGHT = 640

	dst = np.array([
		[0, 0],
		[WIDTH - 1, 0],
		[WIDTH - 1, HEIGHT - 1],
		[0, HEIGHT - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (WIDTH, HEIGHT))
	# return the warped image
	return warped


In [ ]:
def scan_document(input_img):
  # Convert image to gray and blur it
  input_grey = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
  input_grey = cv2.GaussianBlur(input_grey, (5,5), 0)
  ret, input_grey = cv2.threshold(input_grey, 130, 200, cv2.THRESH_BINARY)
  

  # cv2.imshow("BINARY", input_grey)

  img_copy = input_img.copy()

  edges = cv2.Canny(input_grey, 50, 150)
  # to make the edges thick
  kernel = np.ones((5,5), np.uint8)
  # edges = cv2.dilate(edges, kernel, iterations=1)

  corners = get_corners(edges, input_img)

  if corners is None:
    print("INVALID CORNERS")
  else: 
    print("FINAL CORNERS: ", corners)

    # draw corner points on the image
    for x, y in corners:
      cv2.circle(input_img, (int(x), int(y)),10, (255, 0, 0), 4)
    
    top_left = corners[0]
    bottom_left = corners[1]
    top_right = corners[2]
    bottom_right = corners[3]


    cv2.line(input_img, (int(top_left[0]), int(top_left[1])), (int(top_right[0]), int(top_right[1])), (0,0,255), 10, cv2.LINE_AA )

    cv2.line(input_img, (int(top_right[0]), int(top_right[1])), (int(bottom_right[0]), int(bottom_right[1])), (0,0,255), 10, cv2.LINE_AA )

    cv2.line(input_img, (int(bottom_right[0]), int(bottom_right[1])), (int(bottom_left[0]), int(bottom_left[1])), (0,0,255), 10, cv2.LINE_AA )

    cv2.line(input_img, (int(bottom_left[0]), int(bottom_left[1])), (int(top_left[0]), int(top_left[1])), (0,0,255), 10, cv2.LINE_AA )

      
  
    cv2.imshow("LINES", input_img)
    cv2.waitKey(10)

    # perspective_transform(corners)
    warped_img = four_point_transform(img_copy, corners)
    # cv2.imshow("warped img", warped_img)
    # cv2.waitKey(10)
    return warped_img



In [ ]:
# rng.seed(12345)

# # read and show the input image
# img_path = 'images/document-aligned.jpeg'
# input_img = cv2.imread(img_path)
# warped_img = scan_document(input_img)

# cv2.imshow("Corners", input_img)

# if warped_img is not None: 
#   cv2.imshow("Transformation", warped_img)

# cv2.waitKey(10)
# cv2.destroyAllWindows()


In [ ]:
video = cv2.VideoCapture('videos/document-video-compress.mp4')
 
# video.set(cv2.CAP_PROP_FPS, 1)
# Loop until the end of the video
while (video.isOpened()):
 
    # Capture frame-by-frame
    ret, frame = video.read()
    # frame = cv2.resize(frame, (540, 380), fx = 0, fy = 0,
    #                      interpolation = cv2.INTER_CUBIC)
 
    # Display the resulting frame
    cv2.imshow('Video', frame)

    warped_img = scan_document(frame)
    # if warped_img is not None: 
    #     cv2.imshow("Warp", warped_img)
 
# release the video capture object
video.release()
# Closes all the windows currently opened.
cv2.destroyAllWindows()